In [60]:
import numpy as np 
import pandas as pd
import pulp

In [ ]:
"""
Parameters:
h: unit holding cost
p: production capacity per month
I_0: initial_inventory
c_t: unit production cost in month t
d_t: demand of month t

Variables:
P_t: Amount produced in month t
I_t: Inventory at the end of period t

Constraints:
Inventory Constraints: I_{t-1} + X_t - d_t = I_t
Capacity Constraints: P_t <= p

Objective: Min Sum(h*I_t + c_t*X_t)
"""

In [61]:
filepath = '/Volumes/Elements/khartless/Projects/optimization-tutorial/data/csv/'
input_name = 'input_data.csv'
parameters_name = 'parameters.csv'

input_data = pd.read_csv(filepath+input_name, index_col=[0])
parameters = pd.read_csv(filepath+parameters_name, index_col=[0])

In [62]:
input_data

,demand,production_cost,production_capacity
period,,,
1,3300,150,6400
2,3300,150,6400
3,4500,140,6800
4,6100,140,6800
5,5100,150,6400
6,9000,135,7000
7,9700,135,7000
8,8200,135,7000
9,7900,135,6800


In [63]:
parameters

,value
attribute,
holding_cost,8
initial_inventory,500


In [64]:
model = pulp.LpProblem(name='prod_planning', sense=pulp.LpMinimize)

In [65]:
production_variables = pulp.LpVariable.dicts(name='P', indexs=input_data.index,
                                             lowBound=0, cat=pulp.LpContinuous)

In [66]:
inventory_variables = pulp.LpVariable.dicts(name='I', indexs=input_data.index,
                                            lowBound=0, cat=pulp.LpContinuous)

In [67]:
for period, value in input_data.iloc[1:].iterrows():
    model.addConstraint(pulp.LpConstraint(
        e=inventory_variables[period - 1] + production_variables[period] - inventory_variables[period],
        sense=pulp.LpConstraintEQ,
        name='inv_balance' + str(period),
        rhs=value.demand))

In [69]:
# inv balance for first period
model.addConstraint(pulp.LpConstraint(
    e=production_variables[1] - inventory_variables[1],
    sense=pulp.LpConstraintEQ,
    name='inv_balance0',
    rhs=input_data.iloc[1].demand - parameters.loc['initial_inventory'].value))

In [71]:
for index, value in production_variables.items():
    model.addConstraint(pulp.LpConstraint(
        e=value,
        sense=pulp.LpConstraintLE,
        name='prod_cap_month_' + str(index),
        rhs=input_data.iloc[index-1].production_capacity))

In [75]:
total_holding_cost = parameters.loc['holding_cost'].value * pulp.lpSum(inventory_variables)
total_production_cost = pulp.lpSum(row['production_cost'] * production_variables[index]
                                   for index, row in input_data.iterrows())

objective = total_holding_cost + total_production_cost

model.setObjective(objective)

In [78]:

model.solve()


1

In [80]:
print('The solution is optimal and the objective value is ${:,.2f}'.format(pulp.value(model.objective)))

The solution is optimal and the objective value is $10,183,400.00
